In [6]:
# Data Preprocessing:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# #GAN Model:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

import math
import matplotlib.pyplot as plt

from collections import namedtuple
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
# from tensorflow.keras.layers import BatchNormalization, LeakyReLU
# from tensorflow.keras.models import Sequential

# Fraud Detection GAN

In [7]:
#TODO:

#1. Resample Data before adding to discriminator
#2. Add in Generator
#3. Increase Number of Epochs
#4. Analyze ROC and recall-precision curves
#5. Hyperparametrization (if it's even possible)

In [ ]:
device = torch.device("cpu")

In [9]:
fraud_data = pd.read_csv("fraud_data.csv")
fraud_data.drop_duplicates(inplace=True)
fraud_data.Class.value_counts()

Class
0    21204
1      346
Name: count, dtype: int64

In [10]:
# torch.from_numpy(np.array(fraud_data.iloc[0,1]).astype('float32'))#.unsqueeze(dim = 0).size()
lambda x: torch.from_numpy(np.array(x).astype('float32')).unsqueeze(dim=0)

<function __main__.<lambda>(x)>

In [11]:
# y_strat = np.hstack([np.ones(shape = 5000), np.zeros(shape = fraud_data.shape[0]-5000)])
df_train, df_test = train_test_split(fraud_data, test_size=0.25, stratify = fraud_data.Class)
display(df_train.Class.value_counts(normalize = True), df_test.Class.value_counts(normalize = True))

Class
0    0.983975
1    0.016025
Name: proportion, dtype: float64

Class
0    0.983853
1    0.016147
Name: proportion, dtype: float64

In [12]:
#df_train.query("index <= 5000").shape[0]/df_train.query("index > 5000").shape[0]
df_test.query("index > 5000").shape[0]/df_test.shape[0]


0.7702301410541945

In [13]:
PdToTensor = namedtuple("PdToTensor",["df","data","labels"])
m = PdToTensor([4,5,6],[1,2,3],[1,0,0])
m

# transform = torch.from_numpy,
# target_transform = lambda x: torch.from_numpy(np.array(x).astype('float32')).unsqueeze(dim=0),

PdToTensor(df=[4, 5, 6], data=[1, 2, 3], labels=[1, 0, 0])

In [14]:
BATCH_SIZE = 32
PdToTensor = namedtuple("PdToTensor",["df","sample","data","labels"])

class CustomPandasTorch(Dataset):
    def __init__(self, df):
        # self.df = df.astype('float32') 
        self.raw = CustomPandasTorch.transform_data(df)
        self.active = self.raw
        # df_sample = (df
        #     .groupby('Class')
        #     [df.columns]
        #     .apply(lambda x: x.sample(sample_size, random_state = 6, replace = True))
        #     .sample(frac=1, random_state = 6)
        #     .reset_index(drop = True)
        # )
        # self.sample = CustomPandasTorch.transform_data(df_sample)

        df_train, df_test = train_test_split(df, test_size=0.25, stratify = df.Class)
        self.train, self.test = CustomPandasTorch.transform_data(df_train), CustomPandasTorch.transform_data(df_test)

        # self.sample_size = sample_size

        # if transform_type == "rs":
            #self.df = self.pd_sample(self.df)
        #elif transform_type == "split":        
            # self.df, self.df_test = train_test_split(self.df, test_size=0.25, stratify = fraud_data.Class)
            #self.df = self.pd_sample(self.df)
            # self.data_test = self.df_test.iloc[:,:-1]
            # self.label_test = self.df_test.iloc[:,-1]
            
        # self.data, self.label = CustomPandasTorch.extractXy(self.df)

    def transform_data(df_input):
        sample_size = max((df_input.shape[0]//BATCH_SIZE+1)*BATCH_SIZE, 16000)
        df = (df_input
                .groupby('Class')
                [df_input.columns]
                .apply(lambda x: x.sample(sample_size, random_state = 6, replace = True))
                .sample(frac=1, random_state = 6)
                .reset_index(drop = True)        
        )

        t = torch.from_numpy(df.astype('float32').values)
        return PdToTensor(df_input, df, t[:,:-1], t[:,-1:])
    
    def set_raw(self):
        self.active = self.raw
    
    def set_train(self):
        self.active = self.train

    def set_test(self):
        self.active = self.test

    def __len__(self):
        return self.active.sample.shape[0]
    
    def __getitem__(self, idx):
        # data = self.data.iloc[idx,:]
        # label = self.label.iloc[idx]

        # if self.transform:
        #     data = self.transform(data.values)
        # if self.target_transform:
        #     label = self.target_transform(label)
        return self.active.data[idx], self.active.labels[idx]

    # def get_sample(self, size = 1000):
    #     df_resampled = (self.data
    #         .groupby(self.label)
    #         .apply(lambda x: x.sample(size, random_state = 6, replace = True))
    #         .reset_index(level = 0)
    #         .sample(frac=1, random_state = 6)
    #     )

    #     data_rs = self.data.reindex(df_resampled.index)
    #     label_rs = self.label.reindex(df_resampled.index)

    #     return self.transform_data(data_rs, label_rs)
    
    # def pd_sample(self, df):
    #     data = df.iloc[:,:-1]
    #     label = df.iloc[:,-1]

    #     return (data
    #         .groupby(label)
    #         .apply(lambda x: x.sample(self.sample_size, random_state = 6, replace = True))
    #         .reset_index(level = 0)
    #         .sample(frac=1, random_state = 6)
    #         .reindex(columns = df.columns)
    #         # .astype('float32')
    #         .reset_index(drop = True)
    #     )
    
    # def extractXy(df):
    #     return df.iloc[:,:-1], df.iloc[:,-1]

In [15]:
torch.manual_seed(111)

In [16]:
fraud_data.query("Class == 1")#.sample(512, replace = True).reset_index(drop=True)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
15,-1.927453,1.827621,-7.019495,5.348303,-2.739188,-2.107219,-5.015848,1.205868,-4.382713,-8.337707,...,1.376938,-0.792017,-0.771414,-0.379574,0.718717,1.111151,1.277707,0.819081,512.25,1
149,0.432554,1.861373,-4.310353,2.448080,4.574094,-2.979912,-2.792379,-2.719867,-0.276704,-2.314747,...,-1.384477,-0.348904,-3.979948,-0.828156,-2.419446,-0.767070,0.387039,0.319402,1.00,1
182,0.908637,2.849024,-5.647343,6.009415,0.216656,-2.397014,-1.819308,0.338527,-2.819883,-4.063098,...,0.407260,-0.397435,-0.080006,-0.168597,0.465058,0.210510,0.648705,0.360224,1.18,1
255,-22.341889,15.536133,-22.865228,7.043374,-14.183129,-0.463145,-28.215112,-14.607791,-9.481456,-20.949192,...,-9.110423,4.158895,1.412928,0.382801,0.447154,-0.632816,-4.380154,-0.467863,1.00,1
296,-2.880042,5.225442,-11.063330,6.689951,-5.759924,-2.244031,-11.199975,4.014722,-3.429304,-11.561950,...,2.002883,0.351102,0.795255,-0.778379,-1.646815,0.487539,1.427713,0.583172,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,-22.561699,13.208904,-24.643819,6.232532,-16.905611,-4.497439,-16.810184,14.955107,-3.871297,-8.581266,...,1.765987,-1.635517,-0.998317,0.138972,1.559350,-0.222125,1.504425,0.445920,99.99,1
21292,-7.381547,-7.449015,-4.696287,3.728439,6.198304,-6.406267,-5.831452,1.457175,-0.646203,-4.029129,...,1.176575,-0.978692,-0.278330,-0.635874,0.123539,0.404729,0.704915,-1.229992,35.00,1
21325,-3.632809,5.437263,-9.136521,10.307226,-5.421830,-2.864815,-10.634088,3.018127,-4.891640,-11.235048,...,2.309880,0.978660,-0.096130,0.432377,-0.435628,0.650893,1.693608,0.857685,8.54,1
21406,-1.410852,2.268271,-2.297554,1.871331,0.248957,-1.208799,-1.358648,1.102916,-1.317364,-4.626919,...,0.155381,-0.614880,-0.196126,-0.464376,0.118473,-0.484537,0.373596,0.187657,1.00,1


In [17]:
DataGroups = namedtuple("DataGroups", ["all", "fraud", "valid"])
# train_fraud = fraud_data.sample(n = 8000).astype('float32')
# train_fraud = CustomPandasTorch.pd_sample(fraud_data, size = 4000)
# train_fraud_torch = CustomPandasTorch(
#                                         fraud_data.query("Class == 1")#.sample(512, replace = True).reset_index(drop=True)
#                                         # transform = torch.from_numpy,
#                                         # target_transform = lambda x: torch.from_numpy(np.array(x).astype('float32')).unsqueeze(dim=0),
#                                         # transform_type = None
#                                     )

dg = DataGroups(CustomPandasTorch(fraud_data), CustomPandasTorch(fraud_data.query("Class == 1")), CustomPandasTorch(fraud_data.query("Class == 0")))
#TODO: April 2 create 3 train_fraud_torch dataset (DONE)

In [18]:
# train_loader = torch.utils.data.DataLoader(
#     train_fraud_torch, batch_size=BATCH_SIZE, shuffle=True
# )

train_loader = torch.utils.data.DataLoader(
    dg.fraud, batch_size=BATCH_SIZE, shuffle=True
)

In [19]:
train_loader.dataset.set_raw()
data, label = next(iter(train_loader))
display(data.shape, label.shape)

torch.Size([32, 29])

torch.Size([32, 1])

In [20]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(29, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # x = x.view(x.size(0), 784)
        output = self.model(x)
        return output


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 29),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        # output = output.view(x.size(0), 1, 28, 28)
        return output

# discriminator = Discriminator().to(device=device)
# generator = Generator().to(device=device)

In [21]:
lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss()

# optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
# optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [ ]:
def train_discriminator(train_loader):
    lr = 0.0001
    num_epochs = 50
    loss_function = nn.BCELoss()

    discriminator = Discriminator()
    
    optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for n, (real_samples, real_labels) in enumerate(train_loader):
            # Training the discriminator
            discriminator.zero_grad()
            loss_discriminator = loss_function(discriminator(real_samples), real_labels)
            loss_discriminator.backward()
            optimizer_discriminator.step()
            
            # Show loss
            if n == BATCH_SIZE - 1:
                print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")

    return discriminator, None

def train_gan(train_loader):
    lr = 0.0001
    num_epochs = 50
    loss_function = nn.BCELoss()

    discriminator = Discriminator()
    generator = Generator()
    
    optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
    optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for n, (real_samples, real_labels) in enumerate(train_loader):
            # Data for training discriminator
            gen_samples = generator(torch.randn((BATCH_SIZE, 100)))
            gen_labels = torch.zeros((BATCH_SIZE, 1))
            all_samples = torch.cat((real_samples, gen_samples))
            all_labels = torch.cat((real_labels, gen_labels))

            # Training the discriminator
            discriminator.zero_grad()
            loss_discriminator = loss_function(discriminator(all_samples), all_labels)
            loss_discriminator.backward()
            optimizer_discriminator.step()
            
            # Data and training the generator
            generator.zero_grad()
            gen_samples = generator(torch.randn((BATCH_SIZE, 100)))
            loss_generator = loss_function(discriminator(gen_samples), real_labels) 
            loss_generator.backward()
            optimizer_generator.step()

            # Show loss
            if n == BATCH_SIZE - 1:
                print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
                print(f"Epoch: {epoch} Loss G.: {loss_generator}")

    return discriminator, generator


#Recall Calc:
def recall_calc(y_pred, y_test):
    return (((y_pred==1)*(y_test==1)).to(torch.float32).sum())/((y_test==1).to(torch.float32).sum())

# discriminator, generator = train_gan(train_loader)
# recall_calc(discriminator(train_loader.dataset.active.data), train_loader.dataset.active.labels)


In [ ]:
#This cell takes ~4mins to run
result_lst = []

for i, (t, func) in enumerate(zip([dg.all, dg.fraud], [train_discriminator, train_gan])):
    print(i)
    train_loader = torch.utils.data.DataLoader(t, batch_size=BATCH_SIZE, shuffle=True)
    discriminator, generator = func(train_loader)
    result_lst.append([recall_calc(discriminator(t.active.data), t.active.labels), t.active.sample.shape[0], t.active.df.shape[0], discriminator, generator, "all"])
    t.set_train()
    train_loader = torch.utils.data.DataLoader(t, batch_size=BATCH_SIZE, shuffle=True)
    discriminator, generator = func(train_loader)
    result_lst.append([recall_calc(discriminator(t.test.data), t.test.labels), t.active.sample.shape[0], t.active.df.shape[0], discriminator, generator, "train"])
    # break

In [24]:
pd.DataFrame(result_lst, columns = ["recall", "sample_size", "df_size", "disc", "gen", "df_type"])

,recall,sample_size,df_size,disc,gen,df_type
0,tensor(0.8035),43136,21550,Discriminator(\n (model): Sequential(\n (0...,None,all
1,tensor(0.7696),32384,16162,Discriminator(\n (model): Sequential(\n (0...,None,train
2,tensor(0.9937),16000,346,Discriminator(\n (model): Sequential(\n (0...,Generator(\n (model): Sequential(\n (0): L...,all
3,tensor(0.9548),16000,259,Discriminator(\n (model): Sequential(\n (0...,Generator(\n (model): Sequential(\n (0): L...,train


In [25]:
result_lst[3][3]#(t_test.active.data)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=29, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=16, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=16, out_features=1, bias=True)
    (10): Sigmoid()
  )
)

In [26]:
train_idx = dg.fraud.train.df.index
# t_test = CustomPandasTorch(dg.all.active.df.query("index not in @train_idx"))
t_test = CustomPandasTorch(dg.all.raw.df.query("index not in @train_idx"))
recall_calc(result_lst[3][3](t_test.active.data), t_test.active.labels)

tensor(0.9587)

In [ ]:
#Final Recall Calculation with best Discriminator (i.e., results_lst[3][3])

X_test = torch.from_numpy(dg.all.raw.df.query("index not in @train_idx").astype('float32').values)[:,:-1]
y_test = torch.from_numpy(dg.all.raw.df.query("index not in @train_idx").astype('float32').values)[:,-1:]
recall_calc(result_lst[3][3](X_test), y_test)
# display(X_test, y_test)

tensor(0.9655)

In [ ]:
# X_test, y_test = train_loader.dataset.transform_data(*CustomPandasTorch.extractXy(train_loader.dataset.df_test))
# (discriminator(X_test).round().flatten() == y_test).to(torch.float32).mean()

In [29]:
X_test, y_test = next(iter(train_loader))
y_test = y_test.flatten()
print((discriminator(X_test).round().flatten() == y_test).to(torch.float32).mean())
print('-'*50)
y_pred = discriminator(X_test).round().flatten()
#Recall Calculation:
(((y_pred==1)*(y_test==1)).to(torch.float32).sum())/((y_test==1).to(torch.float32).sum())

tensor(1.)
--------------------------------------------------


tensor(1.)

In [ ]:
display(y_pred.shape, y_test.flatten().shape, discriminator(X_test).round().flatten().shape)

torch.Size([32])

torch.Size([32])

torch.Size([32])

In [ ]:
acc_lst = []
for _ in range(10):
    X, y = train_loader.dataset.sample()
    acc_lst.append((discriminator(X).round().flatten() == y).to(torch.float32).mean())
acc_lst

[tensor(0.9660, dtype=torch.float64),
 tensor(0.9690, dtype=torch.float64),
 tensor(0.9680, dtype=torch.float64),
 tensor(0.9620, dtype=torch.float64),
 tensor(0.9645, dtype=torch.float64),
 tensor(0.9660, dtype=torch.float64),
 tensor(0.9655, dtype=torch.float64),
 tensor(0.9620, dtype=torch.float64),
 tensor(0.9710, dtype=torch.float64),
 tensor(0.9630, dtype=torch.float64)]

In [ ]:
#Archive Accuracy list:
#acc_lst0 = acc_lst
# acc_lst1 = [tensor(0.9660, dtype=torch.float64),
#  tensor(0.9690, dtype=torch.float64),
#  tensor(0.9680, dtype=torch.float64),
#  tensor(0.9620, dtype=torch.float64),
#  tensor(0.9645, dtype=torch.float64),
#  tensor(0.9660, dtype=torch.float64),
#  tensor(0.9655, dtype=torch.float64),
#  tensor(0.9620, dtype=torch.float64),
#  tensor(0.9710, dtype=torch.float64),
#  tensor(0.9630, dtype=torch.float64)] 
#  NOTE: post-resample for training data

#acc_test2 = tensor(0.9830)

#Archive Predictions:
#y_pred0 = y_pred

#Archive Recall:
#0: tensor(0.8030, dtype=torch.float64)
#1: tensor(0.9370, dtype=torch.float64) NOTE: post-resample for training data
#2: tensor(0.8652) NOTE: post-resample and train_test_split

[tensor(0.8950, dtype=torch.float64),
 tensor(0.8960, dtype=torch.float64),
 tensor(0.8945, dtype=torch.float64),
 tensor(0.9015, dtype=torch.float64),
 tensor(0.8995, dtype=torch.float64),
 tensor(0.8975, dtype=torch.float64),
 tensor(0.8995, dtype=torch.float64),
 tensor(0.8980, dtype=torch.float64),
 tensor(0.8995, dtype=torch.float64),
 tensor(0.8955, dtype=torch.float64)]

tensor([1., 0., 0.,  ..., 0., 0., 0.], grad_fn=<ViewBackward0>)

In [ ]:
y_pred = discriminator(X).round().flatten()
#Recall Calculation:
(((y_pred==1)*(y==1)).to(torch.float32).sum())/((y==1).to(torch.float32).sum())

tensor(0.9370, dtype=torch.float64)

In [ ]:
y_pred = discriminator(X_test).round().flatten()
#Recall Calculation (Test):
(((y_pred==1)*(y_test==1)).to(torch.float32).sum())/((y_test==1).to(torch.float32).sum())

tensor(0.8652)

# Precision, Accuracy, Recall Visual
![alt text](precision_acc_recall.png "Title")